In [2]:
import sys
sys.path.append('/path/to/ffmpeg')

In [8]:
from pydub import AudioSegment
import numpy as np
import os
import random

# Paths to your clean speech and noise directories
clean_speech_dir = '/Users/anude/Downloads/ARU_Speech_Corpus_v1'
noise_dir = "/Users/anude/Downloads/RIRS_NOISES/pointsource_noises"
output_dir = "/Users/anude/Desktop/output"


# SNR levels to generate (e.g., -5 dB and 0 dB)
snr_levels = [-5, 0]

# Function to mix clean speech with noise at a specific SNR level
def mix_audio(clean_audio, noise_audio, snr):
    # Convert AudioSegment to numpy arrays
    clean_array = np.array(clean_audio.get_array_of_samples())
    noise_array = np.array(noise_audio.get_array_of_samples())
    
    # Calculate the scaling factor for noise to achieve the desired SNR
    snr_ratio = 10 ** (snr / 10)
    
    # Ensure both arrays are the same length
    min_length = min(len(clean_array), len(noise_array))
    clean_array = clean_array[:min_length]
    noise_array = noise_array[:min_length]
    
    # Scale the noise array to match the desired SNR
    scaled_noise_array = clean_array + (noise_array * (1 / snr_ratio))
    
    # Convert the scaled array back to an AudioSegment
    mixed_audio = AudioSegment(scaled_noise_array.tobytes(), frame_rate=clean_audio.frame_rate, sample_width=clean_audio.sample_width, channels=clean_audio.channels)
    
    return mixed_audio

# Iterate through clean speech files
for root, dirs, files in os.walk(clean_speech_dir):
    for file in files:
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
        clean_audio = AudioSegment.from_file(file_path)
        
        # Randomly select a noise file from the noise directory
        noise_file = random.choice(os.listdir(noise_dir))
        noise_audio = AudioSegment.from_file(os.path.join(noise_dir, noise_file), 'wav')

        
        # Generate noisy speech at multiple SNR levels
        for snr in snr_levels:
            mixed_audio = mix_audio(clean_audio, noise_audio, snr)
            
            # Define the output path for the noisy audio
            output_path = os.path.join(output_dir, f"SNR_{snr}dB", file)
            
            # Create the output directory if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # Export the mixed audio as a WAV file
            mixed_audio.export(output_path, format="wav")

print("Noise generation and mixing complete.")


Noise generation and mixing complete.


In [14]:
import shutil

In [13]:
import os
import random
import shutil

# Paths to your data directories
clean_speech_dir = '/Users/anude/Downloads/ARU_Speech_Corpus_v1'
noisy_speech_dir = "/Users/anude/Desktop/output/SNR_0dB"
output_dir = "/Users/anude/Desktop/output"

# Split ratios
train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

# Create directories for train, validation, and test sets
for split in ["train", "validation", "test"]:
    os.makedirs(os.path.join(output_dir, split, "clean"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, split, "noisy"), exist_ok=True)

# Iterate through your data and assign each file to a split
for root, dirs, files in os.walk(clean_speech_dir):
    for file in files:
        clean_audio_path = os.path.join(root, file)
        noisy_audio_path = os.path.join(noisy_speech_dir, file)  # Assuming the noisy data has the same file names

        # Randomly assign the file to a split
        split_choice = random.random()
        if split_choice < train_ratio:
            split = "train"
        elif split_choice < train_ratio + validation_ratio:
            split = "validation"
        else:
            split = "test"

        # Move the clean and noisy audio to the corresponding split directory
        clean_output_path = os.path.join(output_dir, split, "clean", file)
        noisy_output_path = os.path.join(output_dir, split, "noisy", file)
        
        shutil.copy(clean_audio_path, clean_output_path)
        shutil.copy(noisy_audio_path, noisy_output_path)

print("Data splitting and organization complete.")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/anude/Desktop/output/SNR_0dB/.DS_Store'